In [13]:
# load tidy dataset

import pandas as pd
import os.path
import numpy as np
from itertools import combinations
import glob
import sys
import time

#import numpy as np
#import loadTidyCSV.py

def loadTidy(tidyData): 
    assert os.path.isfile(tidyData), "desired file does not exist" 
    df = pd.read_csv(tidyData, header = None, names = ["date", "animal", "session", "trial", "stimulus", "neuronID", "timePt", "CaSignal"])
    return df 

# helper functions to return the number of distinct types in the provided data frame
getNumNeurons = lambda df: len(np.unique(df['neuronID'].tolist()))
getNeurons = lambda df: np.unique(df['neuronID'].tolist())

getNumTrials = lambda df: len(np.unique(df['trial'].tolist()))
getTrials = lambda df: np.unique(df['trial'].tolist())


# pass in pre-filtered data set containing data for only one animal and the same session (ie SAME NEURONS!)
def getListsOfTrialIDs(df_animalSession):
    ## get trials for both stimuli
    df_anmlSessStimA = df_animalSession[df_animalSession['stimulus'] == stimA]
    df_anmlSessStimB = df_animalSession[df_animalSession['stimulus'] == stimB]
    print(stimA,stimB)

    ## get lists of trial numbers of each stimuli's presentations 
    trials_stimA = np.unique(df_anmlSessStimA['trial'].tolist())
    trials_stimB = np.unique(df_anmlSessStimB['trial'].tolist())
    print("trial IDs for each stimulus type",trials_stimA,trials_stimB)
    return (trials_stimA,trials_stimB)

## pass in a data frame with only a single animal and session    
def getNumTimePtsPerTrial(df_animalSession, trials_stimA, trials_stimB):

    #### get number of timePts in each trial selected above 
    ## (1 to 3 presentations of the same stimuli exist per session in Prabhat's data)
    numTimePtsPerTrial = np.empty((2,max(len(trials_stimA),len(trials_stimB))))
    numTimePtsPerTrial[:] = np.nan
    stimInd = 0;
    for thisStimTypeTrialNums in [trials_stimA, trials_stimB]:
        trialInd = 0
        for trial in thisStimTypeTrialNums:
            inds_thisTrial = (df_animalSession['trial']==trial)
            tmp_df_thisTrial = df_animalSession[inds_thisTrial] # gives all time points for all neurons
            numNeurons = getNumNeurons(tmp_df_thisTrial) 
            numTimePtsPerTrial[stimInd,trialInd] = np.sum(inds_thisTrial)/numNeurons 
            trialInd += 1
        stimInd += 1
    print(numTimePtsPerTrial) # rows are for stimuli type; cols are presentation of that stimulus
    return numTimePtsPerTrial

## test candidate comparisons based on whether the number of trials per session and approximate number of timePts match
def areNumTrialsPerStimulusEqual(numTimePtsPerTrial):
    
    ## no trials of either type --> discard this comparison for this animal/session   
    if np.all(np.isnan(numTimePtsPerTrial)):
        print("DISCARDED: neither stimulus type were found for this animal and session")
        return False  # skip to next session (WORK: handle this)
        
    ## different numbers of trials per stimuli/session --> discard this comparison for this animal/session 
    elif np.any(np.isnan(numTimePtsPerTrial)): 
        print("DISCARDED: mismatching numbers of trials per stimulus type for this animal/session")
        return False # skip to next session (WORK: handle this)

    ## FULFILLED here: condition that allows analysis to proceed to attempted data
    elif not np.any(np.isnan(numTimePtsPerTrial)): 
        print("trial numbers match")
    else:
        raise RuntimeError('unexpected trial comparison occurred')
        return False
    
    print("checking approx num of time points")

## input argument generated from getNumTimePtsPerTrial
def areNumTimePtsPerTrialSimilar(numTimePtsPerTrial):
    minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)             
    if (np.abs(minTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))) \
        or (np.abs(maxTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))): 
        print("DISCARDED: variance in trial length is above the user's threshold")
        return False # skip to next session (WORK: handle this)

    ### passed all criteria if it made it this far
    return True

## input argument created by getNumTimePtsPerTrial function
def timePtStats(numTimePtsPerTrial):
    minTPs = int(np.amin(numTimePtsPerTrial))
    maxTPs = int(np.amax(numTimePtsPerTrial))
    meanTPs = np.mean(numTimePtsPerTrial)
    stdTPs = np.std(numTimePtsPerTrial)
    print('min', minTPs)
    print('max', maxTPs)
    print('std', stdTPs)
    print('mean',meanTPs)
    print('|min-mean|=',np.abs(minTPs-meanTPs))
    print('|max-mean|=',np.abs(maxTPs-meanTPs))
    print('|mean-std|=',np.abs(meanTPs-stdTPs))
    print('thresh * |mean-std|=',(threshTPs_stdFromMean * np.abs(meanTPs-stdTPs)))
    return minTPs, maxTPs, meanTPs, stdTPs

In [2]:
################ concatenate all .csv files exported from matlab into single pandas dataframe df

# tidy csv file and dir (use makeTidy_Anderson.m to convert Ann's structure to csv)
tidyDataDir = '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/'
tidyDataFileTemplate = 'mouse'
tidyDataFileExt = '.csv'

# get all input files you want to add to the same dataset
dataFiles = np.sort(glob.glob(\
    "/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse*.csv"))
print("data files: \n", dataFiles)


print("\n loading and appending to prior pandas data frame")
dataLst = []
for file in dataFiles:
    print(file)
    dataLst.append(loadTidy(file))
df = pd.concat(dataLst)

print('finished loading')

############# ALL DATA STORED HERE IN DF
    

data files: 
 [ '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse4.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse5.csv']

 loading and appending to prior pandas data frame
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAn

In [3]:
## survey/search data to prepare for split operation
metaStrs = [['dates','date'],['animals','animal'],['sessions','session'],['maxTrials','trial'],['stimuli','stimulus']]

print("searching over entire data set to get range of various IDs for data (used in subsequent loops)") 

## this dictionary holds useful info regarding the range of inputs to loop/search over subsequently
metaDct = {}
for a,b in metaStrs:
    print(a,b)
    metaDct[a] = np.unique(df[b].tolist())
print(metaDct)


searching over entire data set to get range of various IDs for data (used in subsequent loops)
dates date
animals animal
sessions session
maxTrials trial
stimuli stimulus
{'dates': array(['2017_05_00'], 
      dtype='<U10'), 'animals': array([1, 3, 4, 5, 7]), 'sessions': array([1, 2, 3]), 'maxTrials': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21]), 'stimuli': array(['USS', 'baseline', 'female', 'male', 'mineral oil odor',
       'peanut odor', 'pred odor', 'rat', 'tone', 'toy'], 
      dtype='<U16')}


In [4]:
######### USER PARAMETERS #########
threshTPs_stdFromMean = 1

In [ ]:
# get all pairs of stimuli
stimCmbTpl = tuple(combinations(metaDct['stimuli'],2)) 

######### MAIN LOOP ##########
totalNumComparisons = 0;
for (stimA, stimB) in stimCmbTpl:
    print((stimA,stimB))
    totalNumComparisons += 1
    print("stimuli comparison num: ", totalNumComparisons)
    
    ## get all data for both trial types
    indsBoth = (df['stimulus']==stimA) | (df['stimulus']==stimB)
    df_bothStimuli = df[indsBoth]

    #### select data by animals and sessions
    for animal in metaDct['animals']:
        print('animal: ', animal)
        for session in metaDct['sessions']:
            print('session:', session)
            
            ## return subselection of data where the same neurons were recorded
            inds_animalSession = (df_bothStimuli['animal'] == animal) & (df_bothStimuli['session'] == session)
            df_animalSession = df_bothStimuli[inds_animalSession]
            
            # get lists of trial IDs matching stimuli
            trials_stimA, trials_stimB = getListsOfTrialIDs(df_animalSession)
            
            #### skip this comparison if the data don't match in number of trials or approx number of timePts
            numTimePtsPerTrial = getNumTimePtsPerTrial(df_animalSession,trials_stimA,trials_stimB)
            if areNumTrialsPerStimulusEqual(numTimePtsPerTrial)==False:
                break            
            
            ## truncate longer trials to shortest trial and save to new df
            minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)
            truncLst = []
            for trial in np.concatenate((trials_stimA,trials_stimB)):
                tmp_inds_trunc = (df_animalSession['trial']==trial) & (df_animalSession['timePt'] < minTPs)
                tmp_df_trunc = df_animalSession[tmp_inds_trunc]
                truncLst.append(tmp_df_trunc)
            df_trunc = pd.concat(truncLst)
                    
            ## concatenate same cells 
            neuronArr_anmlSess = np.empty((NUM_ROWS_INITIALIZE,minTPs-1)) # -1 for 0 indexing
            for stimLst in [trials_stimA, trials_stimB]:
                if (stimLst == trials_stimA).all:
                    print('\nstimulus:', stimA)
                elif (stimLst == trials_stimA).all:
                    print('\nstimulus:', stimB)
                else:
                    raise RuntimeError('unexpected trial concatenation condition occurred')
                
                neurons = getNeurons(df_trunc)
                tmp_neuronsArr_sameStim = np.empty((len(neurons),minTPs-1)) # for 0 indexing
                for trial in stimLst:
                    print("appending same neurons in trial: ", trial)
                    
                    tmp_neuronsArr_sameStim_sameTrial = np.empty((getNumNeurons(df_trunc),minTPs-1))
                    for neuron in neurons:
                        tmp_neuronInds = (df_trunc['trial']==trial) & (df_trunc['neuronID']==neuron)
                        tmp_neuronSeries = df_trunc.loc[tmp_neuronInds,'CaSignal']
                        tmp_neuronVec = tmp_neuronSeries.as_matrix()
                        tmp_neuronsArr_sameStim_sameTrial[neuron-1,:] = tmp_neuronVec
                    
                    print('sameStim',np.shape(tmp_neuronsArr_sameStim))
                    print('sameStim sameTrial',np.shape(tmp_neuronsArr_sameStim_sameTrial))
#                     tmp_sameStimNeuronsArr = np.c_[tmp_sameStimNeuronsArr, tmp_neuronsArr_sameStim_sameTrial]
                    tmp_neuronsArr_sameStim = np.concatenate((tmp_neuronsArr_sameStim, tmp_neuronsArr_sameStim_sameTrial), axis=1)
                    print('sameStim',np.shape(tmp_neuronsArr_sameStim))
                
            print('\n')
        print('########\n')

print('total number of comparisons: ', totalNumComparisons)

('USS', 'baseline')
stimuli comparison num:  1
animal:  1
session: 1
USS baseline
trial IDs for each stimulus type [ 5 10 16] [ 1  8 15]
[[ 1117.  1116.  1117.]
 [  370.   363.  1476.]]
trial numbers match
checking approx num of time points
min 363
max 1476
std 415.865663406
mean 926.5
|min-mean|= 563.5
|max-mean|= 549.5
|mean-std|= 510.634336594
thresh * |mean-std|= 510.634336594

stimulus: USS
appending same neurons in trial:  5
sameStim (24, 362)
sameStim sameTrial (24, 362)
sameStim (24, 724)
appending same neurons in trial:  10
sameStim (24, 724)
sameStim sameTrial (24, 362)
sameStim (24, 1086)
appending same neurons in trial:  16
sameStim (24, 1086)
sameStim sameTrial (24, 362)
sameStim (24, 1448)

stimulus: USS
appending same neurons in trial:  1
sameStim (24, 362)
sameStim sameTrial (24, 362)
sameStim (24, 724)
appending same neurons in trial:  8
sameStim (24, 724)
sameStim sameTrial (24, 362)
sameStim (24, 1086)
appending same neurons in trial:  15
sameStim (24, 1086)
sameStim

sameStim (25, 3042)
sameStim sameTrial (25, 1014)
sameStim (25, 4056)

stimulus: USS
appending same neurons in trial:  8
sameStim (25, 1014)
sameStim sameTrial (25, 1014)
sameStim (25, 2028)
appending same neurons in trial:  9
sameStim (25, 2028)
sameStim sameTrial (25, 1014)
sameStim (25, 3042)
appending same neurons in trial:  20
sameStim (25, 3042)
sameStim sameTrial (25, 1014)
sameStim (25, 4056)


session: 3
USS female
trial IDs for each stimulus type [ 6 13 16] [ 3 14 17]
[[ 1139.  1140.  1232.]
 [ 1117.  1151.  1141.]]
trial numbers match
checking approx num of time points
min 1117
max 1232
std 36.6272515425
mean 1153.33333333
|min-mean|= 36.3333333333
|max-mean|= 78.6666666667
|mean-std|= 1116.70608179
thresh * |mean-std|= 1116.70608179

stimulus: USS
appending same neurons in trial:  6
sameStim (21, 1116)
sameStim sameTrial (21, 1116)
sameStim (21, 2232)
appending same neurons in trial:  13
sameStim (21, 2232)
sameStim sameTrial (21, 1116)
sameStim (21, 3348)
appending same ne

USS male
trial IDs for each stimulus type [ 4 11 17] [ 5 12 16]
[[ 1267.  1127.  1123.]
 [ 1123.  1108.  1138.]]
trial numbers match
checking approx num of time points
min 1108
max 1267
std 54.0853235381
mean 1147.66666667
|min-mean|= 39.6666666667
|max-mean|= 119.333333333
|mean-std|= 1093.58134313
thresh * |mean-std|= 1093.58134313

stimulus: USS
appending same neurons in trial:  4
sameStim (30, 1107)
sameStim sameTrial (30, 1107)
sameStim (30, 2214)
appending same neurons in trial:  11
sameStim (30, 2214)
sameStim sameTrial (30, 1107)
sameStim (30, 3321)
appending same neurons in trial:  17
sameStim (30, 3321)
sameStim sameTrial (30, 1107)
sameStim (30, 4428)

stimulus: USS
appending same neurons in trial:  5
sameStim (30, 1107)
sameStim sameTrial (30, 1107)
sameStim (30, 2214)
appending same neurons in trial:  12
sameStim (30, 2214)
sameStim sameTrial (30, 1107)
sameStim (30, 3321)
appending same neurons in trial:  16
sameStim (30, 3321)
sameStim sameTrial (30, 1107)
sameStim (30, 

[[ 1022.  1018.]
 [ 1016.  1009.]]
trial numbers match
checking approx num of time points
min 1009
max 1022
std 4.71036092035
mean 1016.25
|min-mean|= 7.25
|max-mean|= 5.75
|mean-std|= 1011.53963908
thresh * |mean-std|= 1011.53963908

stimulus: USS
appending same neurons in trial:  8
sameStim (40, 1008)
sameStim sameTrial (40, 1008)
sameStim (40, 2016)
appending same neurons in trial:  12
sameStim (40, 2016)
sameStim sameTrial (40, 1008)
sameStim (40, 3024)

stimulus: USS
appending same neurons in trial:  3
sameStim (40, 1008)
sameStim sameTrial (40, 1008)
sameStim (40, 2016)
appending same neurons in trial:  11
sameStim (40, 2016)
sameStim sameTrial (40, 1008)
sameStim (40, 3024)


session: 3
USS male
trial IDs for each stimulus type [ 1 12] [7 9]
[[ 1015.  1053.]
 [ 1022.  1022.]]
trial numbers match
checking approx num of time points
min 1015
max 1053
std 14.7139389696
mean 1028.0
|min-mean|= 13.0
|max-mean|= 25.0
|mean-std|= 1013.28606103
thresh * |mean-std|= 1013.28606103

stimulu

appending same neurons in trial:  1
sameStim (54, 1009)
sameStim sameTrial (54, 1009)
sameStim (54, 2018)
appending same neurons in trial:  12
sameStim (54, 2018)
sameStim sameTrial (54, 1009)
sameStim (54, 3027)

stimulus: USS
appending same neurons in trial:  2
sameStim (54, 1009)
sameStim sameTrial (54, 1009)
sameStim (54, 2018)
appending same neurons in trial:  14
sameStim (54, 2018)
sameStim sameTrial (54, 1009)
sameStim (54, 3027)


########

('USS', 'peanut odor')
stimuli comparison num:  5
animal:  1
session: 1
USS peanut odor
trial IDs for each stimulus type [ 5 10 16] []
[[ 1117.  1116.  1117.]
 [   nan    nan    nan]]
DISCARDED: mismatching numbers of trials per stimulus type for this animal/session
########

animal:  3
session: 1
USS peanut odor
trial IDs for each stimulus type [ 4 11 17] []
[[ 1267.  1127.  1123.]
 [   nan    nan    nan]]
DISCARDED: mismatching numbers of trials per stimulus type for this animal/session
########

animal:  4
session: 1
USS peanut odor
trial

...

sameStim (24, 2230)
sameStim sameTrial (24, 1115)
sameStim (24, 3345)
appending same neurons in trial:  16
sameStim (24, 3345)
sameStim sameTrial (24, 1115)
sameStim (24, 4460)

stimulus: USS
appending same neurons in trial:  2
sameStim (24, 1115)
sameStim sameTrial (24, 1115)
sameStim (24, 2230)
appending same neurons in trial:  12
sameStim (24, 2230)
sameStim sameTrial (24, 1115)
sameStim (24, 3345)
appending same neurons in trial:  19
sameStim (24, 3345)
sameStim sameTrial (24, 1115)
sameStim (24, 4460)


session: 2
USS tone
trial IDs for each stimulus type [ 4 11 21] [ 6 14 17]
[[ 1015.  1329.  1214.]
 [ 1137.  1113.  1127.]]
trial numbers match
checking approx num of time points
min 1015
max 1329
std 96.802921214
mean 1155.83333333
|min-mean|= 140.833333333
|max-mean|= 173.166666667
|mean-std|= 1059.03041212
thresh * |mean-std|= 1059.03041212

stimulus: USS
appending same neurons in trial:  4
sameStim (25, 1014)
sameStim sameTrial (25, 1014)
sameStim (25, 2028)
appending same neur


stimulus: USS
appending same neurons in trial:  6
sameStim (36, 1114)
sameStim sameTrial (36, 1114)
sameStim (36, 2228)
appending same neurons in trial:  9
sameStim (36, 2228)
sameStim sameTrial (36, 1114)
sameStim (36, 3342)

stimulus: USS
appending same neurons in trial:  1
sameStim (36, 1114)
sameStim sameTrial (36, 1114)
sameStim (36, 2228)
appending same neurons in trial:  12
sameStim (36, 2228)
sameStim sameTrial (36, 1114)
sameStim (36, 3342)


session: 2
USS tone
trial IDs for each stimulus type [3] [8]
[[ 1251.]
 [ 1119.]]
trial numbers match
checking approx num of time points
min 1119
max 1251
std 66.0
mean 1185.0
|min-mean|= 66.0
|max-mean|= 66.0
|mean-std|= 1119.0
thresh * |mean-std|= 1119.0

stimulus: USS
appending same neurons in trial:  3
sameStim (35, 1118)
sameStim sameTrial (35, 1118)
sameStim (35, 2236)

stimulus: USS
appending same neurons in trial:  8
sameStim (35, 1118)
sameStim sameTrial (35, 1118)
sameStim (35, 2236)


session: 3
USS tone
trial IDs for each sti

sameStim (35, 2679)
sameStim sameTrial (35, 893)
sameStim (35, 3572)

stimulus: USS
appending same neurons in trial:  7
sameStim (35, 893)
sameStim sameTrial (35, 893)
sameStim (35, 1786)
appending same neurons in trial:  10
sameStim (35, 1786)
sameStim sameTrial (35, 893)
sameStim (35, 2679)
appending same neurons in trial:  21
sameStim (35, 2679)
sameStim sameTrial (35, 893)
sameStim (35, 3572)


session: 3
USS toy
trial IDs for each stimulus type [ 3 14 17] [ 2 12 20]
[[ 1019.  1087.  1156.]
 [  979.  1053.  1004.]]
trial numbers match
checking approx num of time points
min 979
max 1156
std 58.7811383202
mean 1049.66666667
|min-mean|= 70.6666666667
|max-mean|= 106.333333333
|mean-std|= 990.885528346
thresh * |mean-std|= 990.885528346

stimulus: USS
appending same neurons in trial:  3
sameStim (34, 978)
sameStim sameTrial (34, 978)
sameStim (34, 1956)
appending same neurons in trial:  14
sameStim (34, 1956)
sameStim sameTrial (34, 978)
sameStim (34, 2934)
appending same neurons in tr

sameStim (24, 362)
sameStim sameTrial (24, 362)
sameStim (24, 724)
appending same neurons in trial:  11
sameStim (24, 724)
sameStim sameTrial (24, 362)
sameStim (24, 1086)
appending same neurons in trial:  20
sameStim (24, 1086)
sameStim sameTrial (24, 362)
sameStim (24, 1448)


session: 2
baseline female
trial IDs for each stimulus type [ 1  7 15] [ 8  9 20]
[[  689.   681.   685.]
 [ 1116.  1120.  1114.]]
trial numbers match
checking approx num of time points
min 681
max 1120
std 215.852894866
mean 900.833333333
|min-mean|= 219.833333333
|max-mean|= 219.166666667
|mean-std|= 684.980438467
thresh * |mean-std|= 684.980438467

stimulus: baseline
appending same neurons in trial:  1
sameStim (25, 680)
sameStim sameTrial (25, 680)
sameStim (25, 1360)
appending same neurons in trial:  7
sameStim (25, 1360)
sameStim sameTrial (25, 680)
sameStim (25, 2040)
appending same neurons in trial:  15
sameStim (25, 2040)
sameStim sameTrial (25, 680)
sameStim (25, 2720)

stimulus: baseline
appending sa

appending same neurons in trial:  1
sameStim (21, 671)
sameStim sameTrial (21, 671)
sameStim (21, 1342)
appending same neurons in trial:  8
sameStim (21, 1342)
sameStim sameTrial (21, 671)
sameStim (21, 2013)
appending same neurons in trial:  15
sameStim (21, 2013)
sameStim sameTrial (21, 671)
sameStim (21, 2684)

stimulus: baseline
appending same neurons in trial:  4
sameStim (21, 671)
sameStim sameTrial (21, 671)
sameStim (21, 1342)
appending same neurons in trial:  11
sameStim (21, 1342)
sameStim sameTrial (21, 671)
sameStim (21, 2013)
appending same neurons in trial:  19
sameStim (21, 2013)
sameStim sameTrial (21, 671)
sameStim (21, 2684)


########

animal:  3
session: 1
baseline male
trial IDs for each stimulus type [ 1  8 15] [ 5 12 16]
[[  674.   682.   678.]
 [ 1123.  1108.  1138.]]
trial numbers match
checking approx num of time points
min 674
max 1138
std 222.68045117
mean 900.5
|min-mean|= 226.5
|max-mean|= 237.5
|mean-std|= 677.81954883
thresh * |mean-std|= 677.81954883

s

In [18]:
start = np.zeros((24,350))
toAdd = np.zeros((24,350))
np.c_[start, toAdd]

In [ ]:
#                     tmp_trialInds = (data['trial']==trial) 
#                     print(np.sum(tmp_trialInds))
#                     tmp_mat = data[tmp_trialInds].as_matrix()
#                     print(tmp_mat)
#                     print(type(tmp_neuronConcatArr))
#                     np.append(tmp_neuronConcatArr,tmp_mat)
#                     print(tmp_neuronConcatArr)

In [ ]:
testInds = (df['animal']==1) & (df['session']==1) & (df['stimulus']=='USS') & (df['trial']==5)
print(np.sum(testInds))

In [ ]:
np.unique(df_anmlSessStimA[df_anmlSessStimA['trial'] == trialA].tolist())

In [ ]:

# gb = df.groupby(['stimulusType']).get_group('rat')

stimGrouped = df.groupby('stimulusType')
ratGrpd = stimGrouped.get_group('rat')

print(ratGrpd)

print('reached end of file')

gb = df.groupby('stimulusType')

df_means = gb.apply(np.mean)
df_means

for stimType in gb

In [ ]:
ratData = df.loc[df['stimulusType'] == 'rat',:]
ussData = df.loc[df['stimulusType'] == 'USS',:]

print(ussData)

# slicing
df_big_force = df.loc[df['impact force (mN)'] > 1000, :]

df = pd.concat((df_low, df_high), axis=1)

# Specify indices we want (note parentheses holding each Boolean)
inds = (df['food density'] == 'high') & (df['cross-sectional area (sq micron)'] > 2000)

# Pull out areas
df.loc[inds, 'cross-sectional area (sq micron)']

df.corr()


# Rename the impact force column
df = df.rename(columns={'impact force (mN)': 'impf'})



# Write out DataFrame
df.to_csv('xa_combined.csv', index=False)